In [1]:
# Importing all the dependencies 
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

# Data Collection and Pre-Processing 
movies_data = pd.read_csv('movies.csv')

# Selecting the relevant features for recommendation
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director']

# Replacing the missing values with null string
for feature in selected_features:
    movies_data[feature] = movies_data[feature].fillna('')

# Combining all the 5 features
combined_features = (
    movies_data['genres'] + ' ' + 
    movies_data['keywords'] + ' ' + 
    movies_data['tagline'] + ' ' + 
    movies_data['cast'] + ' ' + 
    movies_data['director']
)

# Converting the text data to feature vectors
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)

# Co-Sine similarity
similarity = cosine_similarity(feature_vectors)

# Prepare model data
model_data = {
    'vectorizer': vectorizer,
    'similarity': similarity,
    'movies_data': movies_data
}
# movies_data.dtypes

In [2]:
# Saving the trained model to a .sav file
filename = 'movie_recommendation_model.sav'
pickle.dump(model_data, open(filename, 'wb'))

# Function to recommend movies
def recommend_movie(movie_name):
    # Load the model from .sav file
    model_data = pickle.load(open('movie_recommendation_model.sav', 'rb'))
    vectorizer = model_data['vectorizer']
    similarity = model_data['similarity']
    movies_data = model_data['movies_data']
    
    # Creating the list with all the movie names given in the dataset
    list_of_all_titles = movies_data['title'].tolist()

    # Finding the close match for the movie name given by the user
    find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
    
    if not find_close_match:
        print("Movie not found. Please check the spelling and try again.")
        return
    
    close_match = find_close_match[0]

    # Finding the index of the movie with title
    index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

    # Getting the list of similar movies
    similarity_score = list(enumerate(similarity[index_of_the_movie]))

    # Sorting the movies based on their similarity score
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    print('Movies suggested for you: \n')
    i = 1
    for movies in sorted_similar_movies:
        index = movies[0]
        title_from_index = movies_data[movies_data.index == index]['title'].values[0]
        if i < 11:  # Show top 10 recommendations
            print(i, '.', title_from_index)
            i += 1


In [3]:
# Main function to get the movie name from the user and provide recommendations
def main_menu():
    print("Welcome to the Movie Recommendation System!")
    movie_name = input("Enter your favourite movie name: ")
    recommend_movie(movie_name)

# Call the main menu function
main_menu()

Welcome to the Movie Recommendation System!
Movies suggested for you: 

1 . Charly
2 . Flashdance
3 . The Amazing Spider-Man
4 . The Princess and the Frog
5 . Silent Running
6 . Escape from L.A.
7 . The Helix... Loaded
8 . Spider-Man
9 . The Brain That Wouldn't Die
10 . The Box
